# Join Flags for 2016-2020 joining
In SQL server, an initial join was done between the 2020 employment inventory table and the 2016 table based on the LOCNUM field of 2020 = INFOID_16 field of 2016. From this, most records had a match.

## Join methodology
Join SQL File - https://github.com/SACOG/emp-inventory/blob/main/EMP2020/SQL/Join16_20_Tests_SG.sql

Steps in SQL:
1. Join 2016 to 2020 based on LOCNUM = INFOID_16
2. If there was no 2016 match to a 2020 record after doing the ID-based join, then fill in missing 2016 values based on company name and street addresses both having exact matches (i.e., address2020 = address2016 and bizname2020 = bizname2016).

The resulting table still has rows where there's 2020 but not a corresponding 2016 value, and should still have a 2016 value but do not because the address or biz name changed slightly between the two years. One role of this script is to identify, through the `fuzzywuzzy` python library, where such cases are.

### Fuzzy match methodology
After joining based on matching IDs and having *exact* matches between name and address (if ID doesn't match), we still need to see if there's a fuzzy match. To do this, this script does the following for each:
1. Compare the 2020 name and 2020 address against all names and addresses int the 2016 table
2. Gets the average match, (name match + addr match) / 2, for all instances where both name match and address match ratio > 80. If both are not > 80, then return zero.
3. From the instances where there is an average match, return the 2016 row with the highest average match. This row's name and address become the name and address for the coname16 and addr16 values in the master table.

## Fields considered
* LOCNUM / INFOID_16 - the "unique ID" fields for the respective years
* Business name (coname / coname16) field
* Biz address field (staddr / staddr16)

## Possible check results
* FullExMatch = the LOCNUM in 2020 has a matching INFOID_16 value, and the biz name and address are an EXACT match
* FullFzMatch = the LOCNUM in 2020 has a matching INFOID_16 value, and the biz name and address are a FUZZY match (`fuzz.ratio` > 80)
* IDMatchNameChg = the IDs match between the two years, but the biz name changed
* IDMatchAddrChg = the IDs match between the two years, but the biz address changed
* IDMatchNameAddrChg = the IDs match between the two years, but the biz address and the biz name changed
* NamAddrExMatch = IDs do not match between 2016 and 2020, but the business name and address are an EXACT match
* NamAddrFzMatch = IDs do not match between 2016 and 2020, but the business name and address are a FUZZY match (`fuzz.ratio` > 80)
* NoMatch = The IDs do not match, nor is there a FUZZY match between both the name and address



In [1]:
# define key variables and parameters
in_csv2020 = r"\\data-svr\Monitoring\Employment Inventory\Employment 2020\SQL\DupeFlagAll20210416.csv" # r"C:\Users\dconly\Desktop\Temporary\temp_csv\sutter_emptest_2020.csv"  # 
in_csv2016 = r"\\data-svr\Monitoring\Employment Inventory\Employment 2016\FINAL 2016 EMP FILE\REGION_EMP16_FINAL_060217-wkg.csv" # r"C:\Users\dconly\Desktop\Temporary\temp_csv\sutter_emptest_2016.csv"  # 

match_threshold = 80 # if fuzzy match below this number, then flag the values as being different

# fields from the 2020 table
fld_coname20 = 'coname'
fld_locnum20 = 'locnum'
fld_staddr20 = 'staddr'
fld_zip = 'zip'
fld_naics = 'naics'
fld_naicsd = 'naicsd'
fld_home = 'home'
fld_locemp20 = 'locemp'
fld_latitude = 'latitude'
fld_longitude = 'longitude'
fld_geo_level = 'geo_level'
fld_naics4 = 'naics4'
fld_dupe_flag = 'dupe_flag'
fld_latlon_uid = 'latlon_uid'
fld_coname16 = 'coname16'
fld_staddr16 = 'staddr16'
fld_emp16 = 'emp16'
fld_notes16 = 'notes16'
fld_infoid16 = 'infoid16'

# fields from the 2016 table
fld16_id = 'INFOID_16'
fld16_coname = 'NAME'
fld16_staddr = 'ADDRESS'
fld16_zip = 'ZIP'

# other fields that get added in this script
fld_jflag = 'join_flag'






In [2]:
# Load and set up master 2020 table and flag function

import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz, process

df = pd.read_csv(in_csv2020)
df16 = pd.read_csv(in_csv2016, usecols = [fld16_id, fld16_coname, fld16_staddr])

df[fld_jflag] = '_'

df[fld_locnum20] = df[fld_locnum20].astype(float)
df[fld_infoid16] = df[fld_infoid16].astype(float)





In [17]:
df.columns

Index(['coname', 'locnum', 'staddr', 'zip', 'naics', 'naicsd', 'home',
       'locemp', 'latitude', 'longitude', 'geo_level', 'naics4', 'dupe_flag',
       'latlon_uid', 'coname16', 'staddr16', 'emp16', 'notes16', 'infoid16',
       'join_flag'],
      dtype='object')

In [3]:
# Find and flag records that have ID match in both years but whose names and address significantly differ

"""
* FullExMatch = the LOCNUM in 2020 has a matching INFOID_16 value, and the biz name and address are an EXACT match
* FullFzMatch = the LOCNUM in 2020 has a matching INFOID_16 value, and the biz name and address are a FUZZY match (`fuzz.ratio` > 80)
* IDMatchNameChg = the IDs match between the two years, but the biz name changed
* IDMatchAddrChg = the IDs match between the two years, but the biz address changed
* IDMatchNameAddrChg = the IDs match between the two years, but the biz address and the biz name changed
* NamAddrExMatch = IDs do not match between 2016 and 2020, but the business name and address are an EXACT match
* NamAddrFzMatch = IDs do not match between 2016 and 2020, but the business name and address are a FUZZY match (`fuzz.ratio` > 80)
* NoMatch = The IDs do not match, nor is there a FUZZY match between both the name and address
"""

def avg_match(row, srchname, srchaddr):
    
    row_name = str(row[fld16_coname])
    row_addr = str(row[fld16_staddr])
    
    try:
        fuzzname = fuzz.ratio(row_name, srchname)
        fuzzaddr = fuzz.ratio(row_addr, srchaddr)
    except:
        import pdb; pdb.set_trace()
    
    if fuzzname > match_threshold and fuzzaddr > match_threshold:
        output = (fuzzname + fuzzaddr) / 2
    else:
        output = 0
    
    return output
    
    

def get_fuzzy_matches(in_row, search_df):
    '''If 2016 values for name and address are not in the 2020 table, then do a
    fuzzy match between the 2016 name and 2020 anme, and 2016 address and 2020 address'''
    
    name1 = in_row[fld_coname20]
    addr1 = in_row[fld_staddr20]
    
    temp_df = search_df
    
    fld_avg_fuzzmatch = 'fuzzmatch_avg'
    
    temp_df[fld_avg_fuzzmatch] = temp_df.apply(lambda x: avg_match(x, name1, addr1), axis=1)
    max_match = temp_df[fld_avg_fuzzmatch].max()
    
    # if the max match is zero, then pass. It means there is not a fuzzy match
    if max_match == 0:
        return None
    
    # return row with highest overall match, if the match > 0
    # import pdb; pdb.set_trace()
    temp_df = temp_df.loc[temp_df[fld_avg_fuzzmatch] == max_match].iloc[0]
    
    name_fzmatch = temp_df[fld16_coname]
    addr_fzmatch = temp_df[fld16_staddr]
    
    return {'name': name_fzmatch, 'addr': addr_fzmatch}
        
    
    

def get_jflag_1(in_row):
    
    jflag_fullmatch = 'FullExMatch'
    jflag_idfuzzmatch = 'FullFzMatch'
    jflag_newname = "IDMatchNameChg" 
    jflag_newaddr = "IDMatchAddrChg" 
    jflag_nmaddrchg = "IDMatchNameAddrChg" 
    jflag_nmaddrematch = 'NamAddrExMatch' 
    jflag_nmaddrfmatch = 'NamAddrFzMatch' 
    jflag_nomatch16 = 'NotMatch16' 
    
    id16 = in_row[fld_infoid16]
    id20 = in_row[fld_locnum20]
    name16 = str(in_row[fld_coname16])
    name20 = str(in_row[fld_coname20])
    addr16 = str(in_row[fld_staddr16])
    addr20 = str(in_row[fld_staddr20])
    
    id_match = id16 == id20
    name_addr_ematch = name16 == name20 and addr16 == addr20
    id_match_name_fmatch = id_match and fuzz.ratio(name20, name16) > match_threshold
    id_match_addr_fmatch = id_match and fuzz.ratio(addr20, addr16) > match_threshold
    
    # if there's a fuzzy match for name and address, populate the name and address 2016 fields in the master df
    
    fuzz_dict = None
    
    # comparing "20" fields to "16" fieles within master table (not comparing to or looking at 2016 table)
    
    if id_match:
        name_fuzzmatch = fuzz.ratio(name20, name16) > match_threshold
        addr_fuzzmatch = fuzz.ratio(addr20, addr16) > match_threshold
        
        if name_addr_ematch:
            output = jflag_fullmatch
        elif name_fuzzmatch and addr_fuzzmatch: # if there's fuzzy match between name and address, and id match
            output = jflag_idfuzzmatch
        elif name_fuzzmatch and not addr_fuzzmatch: # address changed, and id match
            output = jflag_newaddr
        elif addr_fuzzmatch and not name_fuzzmatch: # biz name changed, and id match
            output = jflag_newname
        elif not addr_fuzzmatch and not name_fuzzmatch: # biz name and address changed, and id match
            output = jflag_nmaddrchg
        else:
            output = 'ERROR'
    else:
        if name_addr_ematch:
            output = jflag_nmaddrematch # no id match, but exact name and address match
        else:
            fuzz_dict = get_fuzzy_matches(in_row, df16)
            
            if fuzz_dict: # if there's a fuzzy match for 2016 in the 2016 table, then update the "16" fields in the master table
                df.loc[df[fld_locnum20] == id20, fld_coname16] = fuzz_dict['name']
                df.loc[df[fld_locnum20] == id20, fld_staddr16] = fuzz_dict['addr'] 
                
                output = jflag_nmaddrfmatch # no id match, but fuzzy name and address match to 2016 (requires looking up to 2016 table)
            else:
                output = jflag_nomatch16
    
    return output
        
        
        
        
        

In [ ]:
# apply the function above to calculate each record's join flag
%timeit df[fld_jflag] = df.apply(lambda x: get_jflag_1(x), axis=1)


In [4]:
# TEST CELL, DELETE WHEN DONE (4/27/2021)
df_test = df.loc[df[fld_coname20] == 'SUTTER ROSEVILLE MEDICAL CTR']

In [6]:
dft1 = df_test.iloc[0]
dft1

coname                SUTTER ROSEVILLE MEDICAL CTR
locnum                                 1.45592e+06
staddr                          1 MEDICAL PLAZA DR
zip                                          95661
naics                                     62211002
naicsd        General Medical & Surgical Hospitals
home                                           NaN
locemp                                        1442
latitude                                   38.7651
longitude                                  -121.25
geo_level                                        P
naics4                                        6221
dupe_flag                                      DMN
latlon_uid                                      14
coname16                                       NaN
staddr16                                       NaN
emp16                                          NaN
notes16                                        NaN
infoid16                                       NaN
join_flag                      

In [31]:
df[fld_jflag].value_counts()

# df.info()


# df.loc[(df[fld_jflag] == 'No16ID') & (pd.notnull(df['locnum'])) & (pd.notnull(df['infoid16']))][testcols].head()

# dft = df.loc[df[fld_locnum20].isin([104833801, 403881922])]
# print(dft.iloc[0]['locnum'])
# print(dft.iloc[0]['infoid16'])
# print(dft.iloc[0]['infoid16'] - dft.iloc[0]['locnum'])
# dft['lnum_str']

NotMatch16            296274
FullExMatch            77848
FullFzMatch            28154
IDMatchAddrChg          5417
IDMatchNameChg          5110
NamAddrExMatch          3686
IDMatchNameAddrChg       616
Name: join_flag, dtype: int64

In [32]:
df.to_csv(r"\\data-svr\Monitoring\Employment Inventory\Employment 2020\SQL\RecsAll_w_2016jnflag_20210427.csv", index=False)